In [56]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [57]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

from scipy.optimize import minimize

In [58]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [59]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [60]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
algorithm_name = 'xgbcv'
feature_name = 'stacking2'
model_name = f'{algorithm_name}_{feature_name}_1'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [62]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [63]:
def load_data(model_names, oof_list, test_list, feature_names=None):
    for model in model_names:
        oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
        test_list.append(np.loadtxt(test_dir / f'{model}_test_pred.csv',delimiter=','))
        if feature_names != None:
            feature_names += {f'{model}_class0',f'{model}_class1',f'{model}_class2'}

In [64]:
all_oof = []
all_test = []
feature_names = []

model_names = ['xgbcv_stacking1_1', 'xgbcv_stacking1_2', 'xgbcv_stacking1_3', 'xgbcv_stacking1_4','xgbcv_stacking1_5', 'xgbcv_stacking1_6', 'xgbcv_stacking1_7', 'xgbcv_stacking1_8',
               'lgbmcv_stacking1_1','lgbmcv_stacking1_2','lgbmcv_stacking1_3','lgbmcv_stacking1_4',
               'rfcv_stacking1_1','lrcv_stacking1_1','gbcv_stacking1_1','etscv_stacking1_1','adacv_stacking1_1']
load_data(model_names,all_oof, all_test, feature_names)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)
all_oof.shape, all_test.shape, feature_names

((319923, 51),
 (80000, 51),
 ['xgbcv_stacking1_1_class2',
  'xgbcv_stacking1_1_class0',
  'xgbcv_stacking1_1_class1',
  'xgbcv_stacking1_2_class0',
  'xgbcv_stacking1_2_class2',
  'xgbcv_stacking1_2_class1',
  'xgbcv_stacking1_3_class1',
  'xgbcv_stacking1_3_class2',
  'xgbcv_stacking1_3_class0',
  'xgbcv_stacking1_4_class0',
  'xgbcv_stacking1_4_class2',
  'xgbcv_stacking1_4_class1',
  'xgbcv_stacking1_5_class0',
  'xgbcv_stacking1_5_class1',
  'xgbcv_stacking1_5_class2',
  'xgbcv_stacking1_6_class2',
  'xgbcv_stacking1_6_class0',
  'xgbcv_stacking1_6_class1',
  'xgbcv_stacking1_7_class2',
  'xgbcv_stacking1_7_class0',
  'xgbcv_stacking1_7_class1',
  'xgbcv_stacking1_8_class1',
  'xgbcv_stacking1_8_class2',
  'xgbcv_stacking1_8_class0',
  'lgbmcv_stacking1_1_class0',
  'lgbmcv_stacking1_1_class1',
  'lgbmcv_stacking1_1_class2',
  'lgbmcv_stacking1_2_class1',
  'lgbmcv_stacking1_2_class0',
  'lgbmcv_stacking1_2_class2',
  'lgbmcv_stacking1_3_class2',
  'lgbmcv_stacking1_3_class1',
  '

In [65]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [66]:
# Xgboost
stacking_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 3,
    'random_state': 2020,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor'
}

In [67]:
mlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain,'train'), (dval, 'val')]

    xgb_clf = xgb.train(stacking_params, dtrain, 5000, evals=watchlist, early_stopping_rounds=50, verbose_eval=5000)

    dtest = xgb.DMatrix(all_test)
    stk_test_pred += xgb_clf.predict(dtest) / n_splits
    stk_oof_pred[val_idx] = xgb_clf.predict(dval)
    mlogloss.append(xgb_clf.best_score)

print('mean logloss= ', np.mean(mlogloss))

[0]	train-mlogloss:0.755688	val-mlogloss:0.757592
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[33]	train-mlogloss:0.141198	val-mlogloss:0.152642

[0]	train-mlogloss:0.75584	val-mlogloss:0.755807
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[33]	train-mlogloss:0.141221	val-mlogloss:0.150034

[0]	train-mlogloss:0.755786	val-mlogloss:0.756228
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[34]	train-mlogloss:0.140863	val-mlogloss:0.150583

[0]	train-mlogloss:0.755683	val-mlogloss:0.755642
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss h

# Ensemble

- cv score가 안정적이여서 Stacking 예측값와 함께 가중치 최적화함
- 가중치 최적화에는 scipy의 minimize 사용

In [68]:
def log_loss_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, oof_predictions):
        final_prediction += weight*prediction

    return log_loss(y, final_prediction)

In [69]:
oof_predictions = []
test_predictions = []

model_names = ['xgbcv_stacking1_1', 'xgbcv_stacking1_2', 'xgbcv_stacking1_3', 'xgbcv_stacking1_4','xgbcv_stacking1_5', 'xgbcv_stacking1_6', 'xgbcv_stacking1_7', 'xgbcv_stacking1_8',
               'lgbmcv_stacking1_1','lgbmcv_stacking1_2','lgbmcv_stacking1_3','lgbmcv_stacking1_4',
               'rfcv_stacking1_1','lrcv_stacking1_1','gbcv_stacking1_1','etscv_stacking1_1','adacv_stacking1_1']
load_data(model_names,oof_predictions, test_predictions)

oof_predictions.append(stk_oof_pred)
test_predictions.append(stk_test_pred)
len(oof_predictions), oof_predictions, len(test_predictions), test_predictions

(18, [array([[9.99995112e-01, 2.72299508e-06, 2.17522825e-06],
         [4.74168657e-04, 5.37755191e-01, 4.61770654e-01],
         [9.99741137e-01, 1.66911763e-04, 9.19709055e-05],
         ...,
         [9.99995112e-01, 2.72299508e-06, 2.17522825e-06],
         [9.99997854e-01, 1.05149536e-06, 1.07823098e-06],
         [9.99985695e-01, 5.12483257e-06, 9.21028095e-06]]),
  array([[9.99998212e-01, 8.02710645e-07, 9.54504230e-07],
         [1.21305883e-03, 5.26216209e-01, 4.72570747e-01],
         [9.99936461e-01, 4.05087958e-05, 2.30528822e-05],
         ...,
         [9.99998450e-01, 4.60678990e-07, 1.05475306e-06],
         [9.99997735e-01, 1.05631364e-06, 1.15372768e-06],
         [9.99996305e-01, 1.05324204e-06, 2.68697909e-06]]),
  array([[9.99997735e-01, 1.36645406e-06, 8.50908236e-07],
         [1.15951675e-03, 4.71601397e-01, 5.27239144e-01],
         [9.99813139e-01, 8.85812260e-05, 9.83299906e-05],
         ...,
         [9.99998331e-01, 8.53971983e-07, 8.01132956e-07],
      

In [70]:
best_scores = []
weights = []

iteration = 100 # 여러 번 구한 가중치 값 중 Best score를 선택
for i in tqdm(range(iteration)):
    starting_values = np.random.uniform(size=len(oof_predictions))
    bounds = [(0,1)]*len(oof_predictions)
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})

    res = minimize(log_loss_func,
                   starting_values,
                   method = 'SLSQP',
                   bounds = bounds,
                   constraints = cons,
                   options={'maxiter': 500})
    best_scores.append(res['fun'])
    weights.append(res['x'])

weights = weights[np.argmin(best_scores)]
print('\n')
print('Ensemble score: {}'.format(np.min(best_scores)))
print('Best Weights: {}'.format(weights))

100%|██████████| 100/100 [1:17:12<00:00, 46.32s/it]



Ensemble score: 0.15010219973084368
Best Weights: [2.52768417e-02 4.67191944e-02 3.66134706e-04 1.21683601e-01
 1.03114686e-01 4.72595999e-02 8.69551606e-02 9.24965240e-02
 0.00000000e+00 4.33680869e-19 0.00000000e+00 4.33680869e-19
 3.94396496e-02 0.00000000e+00 2.58665112e-01 1.32406622e-01
 0.00000000e+00 4.56168743e-02]


In [71]:
final_prediction = 0

# 가중치 적용
for weight, p in zip(weights, test_predictions):
    final_prediction += weight*p

print(final_prediction)

[[1.32852623e-04 1.63066342e-01 8.36800805e-01]
 [9.99903925e-01 3.07434438e-05 6.53338031e-05]
 [1.19445671e-04 8.95210772e-02 9.10359484e-01]
 ...
 [9.99912420e-01 2.56899232e-05 6.18884258e-05]
 [1.03652910e-04 5.15295036e-02 9.48366849e-01]
 [9.99910960e-01 2.61690667e-05 6.28678151e-05]]


# 제출 파일 및 기타 파일 생성

In [72]:
# # submission 파일 생성

# sub = pd.read_csv(sample_file)
# sub[target_column] = np.argmax(stk_test_pred, axis=1)
# sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

# submission 파일 생성

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(final_prediction, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [73]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [74]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')

In [ ]:
0.9378